<a href="https://colab.research.google.com/github/dattali18/IR_Assignments/blob/main/Assignment.05/IR_05_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [5]:
# download the files from github
sentences_url = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.05/data/sentences/sentences_classify.csv?raw=true"

In [6]:
# download all the file into the same dir as the note book

import urllib.request

urllib.request.urlretrieve(sentences_url, 'sentences.csv')

print("Files downloaded successfully!")

Files downloaded successfully!


In [ ]:
import pandas as pandas

df = pd.read_csv("sentences.csv")

In [ ]:
# Install required libraries
!pip install transformers sentence-transformers torch

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
from sentence_transformers import SentenceTransformer

In [ ]:
# Load BERT model and tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Load SBERT model
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# Function to generate BERT embeddings
def get_bert_embedding(sentence):
    inputs = bert_tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    # Use the [CLS] token embedding as the sentence embedding
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()

In [ ]:
# Function to generate SBERT embeddings
def get_sbert_embedding(sentence):
    return sbert_model.encode(sentence)

In [ ]:
# Generate BERT embeddings
sentences_df['bert_embedding'] = sentences_df['sentence'].apply(get_bert_embedding)

# Generate SBERT embeddings
sentences_df['sbert_embedding'] = sentences_df['sentence'].apply(get_sbert_embedding)

# Display the updated DataFrame
print(sentences_df.head())